# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[HSE][ML][MS][HW09] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [3]:
!pip install catboost

from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier
from typing import Callable, Union, NoReturn, Optional, Dict, Any, List
from scipy.stats import mode


     |████████████████████████████████| 76.1 MB 56 kB/s 


### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [175]:
def gini(x: np.ndarray) -> float:
    un, count = np.unique(x, return_counts = True)
    func = lambda i: i*(1-i)
    return np.sum(func(count/x.size))

    
def entropy(x: np.ndarray) -> float:
    un, count = np.unique(x, return_counts = True)
    func = lambda i: i*np.log2(i)
    return -np.sum(func(count/x.size))


def gain(left_y: np.ndarray, right_y: np.ndarray, criterion: Callable) -> float:
    full_y = np.hstack((left_y, right_y))
    return criterion(full_y) - criterion(left_y)*(left_y.size/full_y.size) \
    - criterion(right_y)*(right_y.size/full_y.size)

In [176]:
class DecisionTreeLeaf:

    def __init__(self, X):

        self.X = X
        un, count = np.unique(X, return_counts = True)
        self.prob = dict(zip(un, count/len(X)))
        self.y = un[np.argmax(count)]
        

class DecisionTreeNode:

    def __init__(self, split_dim, split_value,  left,  right, depth=0):
        self.split_dim = split_dim
        self.split_value = split_value
        self.left = left
        self.right = right
        self.depth=depth

In [210]:
class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):

        self.criterion = globals()[criterion]
        self.max_features = max_features if isinstance(max_features, int) else int(np.sqrt(X.shape[1])) + 1
        self.min_samples_leaf = min_samples_leaf if min_samples_leaf is not None else 0
        self.max_depth = max_depth if max_depth is not None else 1e+12

        shuffle = np.random.choice(range(X.shape[0]), X.shape[0], replace=True)
        self.X, self.oob_X = X[shuffle], X[~shuffle]
        self.y, self.oob_y = y[shuffle], y[~shuffle]

        self.root = None

    def fit(self):
        self.root = self.rec_split(self.X, self.y, 0)

    def rec_split(self, X, y, depth):

        if len(X) > self.min_samples_leaf and depth < self.max_depth:
            gains = []

            if X.shape[1] > self.max_features:
                features = np.random.choice(range(X.shape[1]), self.max_features, replace=False)
            else:
                features = range(X.shape[1])

            for feat in features:
                feat_vals = X[:, feat]
                split_value = 0.5
                left_y, right_y = y[feat_vals < split_value], y[feat_vals >= split_value]
                gain_cur = gain(left_y, right_y, self.criterion)

                gains.append((gain_cur, split_value, feat))

            best_gain, split_value, split_dim = max(gains, key=lambda x: x[0])
            less_mask = X[:, split_dim] < split_value

            if len(X[less_mask]) == 0 or len(X[~less_mask]) == 0:
                return DecisionTreeLeaf(y)

            left = self.rec_split(X[less_mask], y[less_mask], depth+1)
            right = self.rec_split(X[~less_mask], y[~less_mask], depth+1)

            return DecisionTreeNode(split_dim, split_value, left, right)
        return DecisionTreeLeaf(y)


    def find_leaf(self, X, node):
        if type(node) is DecisionTreeLeaf:
            return node.prob
        y_pred = np.empty(X.shape[0], dtype = object)
        mask = X[:, node.split_dim] < node.split_value
        y_pred[mask] = self.find_leaf(X[mask], node.left)
        y_pred[~mask] = self.find_leaf(X[~mask], node.right)
        
        return y_pred

    def predict_proba(self, X):       
        return list(self.find_leaf(X, self.root))
        
    def predict(self, X):
        proba = self.predict_proba(X)
        return [max(p.keys(), key=lambda k: p[k]) for p in proba]

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [211]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):

        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_estimators = n_estimators
        self.trees = []
    
    def fit(self, X, y):

        for i in range(self.n_estimators):
            tree = DecisionTree(X, y, self.criterion, self.max_depth, self.min_samples_leaf, self.max_features)
            tree.fit()
            self.trees.append(tree)
    
    def predict(self, X):
        preds = self.trees[0].predict(X)

        for tree in self.trees[1:]:
            preds = np.vstack((preds, tree.predict(X)))
        
        return mode(preds, axis=0)[0]


### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [212]:
def feature_importance(rfc):

    err_j = np.empty((len(rfc.trees), rfc.trees[0].X.shape[1]))

    for num, tree in enumerate(rfc.trees):
        n, m = tree.oob_X.shape
        err_oob = np.mean(tree.predict(tree.oob_X) == tree.oob_y)        

        for j in range(m):
            new_oob = tree.oob_X.copy()
            random_idx = np.random.choice(range(n), n, replace=False)
            new_oob[:, j] = tree.oob_X[:, j][random_idx]

            err_oob_j = np.mean(tree.predict(new_oob) == tree.oob_y)
            err_j[num][j] = err_oob - err_oob_j

    err_j = np.mean(err_j, axis=0)
    return err_j

def most_important_features(importance, names, k=20):

    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [213]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [-7.2000e-04 -1.5000e-04  1.9909e-01  1.9900e-01  3.9968e-01 -6.2000e-04]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [98]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [99]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [100]:
X, y_age, y_sex, features = read_dataset("/content/drive/MyDrive/vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [191]:
rfc = RandomForestClassifier(n_estimators=10)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.7225725094577553
Most important features:
1. mudakoff
2. 4ch
3. ovsyanochan
4. dayvinchik
5. rhymes
6. styd.pozor
7. rapnewrap
8. iwantyou
9. pravdashowtop
10. bot_maxim
11. xfilm
12. exclusive_muzic
13. leprum
14. pixel_stickers
15. i_des
16. tumblr_vacuum
17. tnt
18. fuck_humor
19. borsch
20. kino_mania


#### Пол

In [197]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.8537200504413619
Most important features:
1. mudakoff
2. 40kg
3. igm
4. rapnewrap
5. girlmeme
6. zerofat
7. cook_good
8. 9o_6o_9o
9. academyofman
10. modnailru
11. reflexia_our_feelings
12. 4ch
13. fuck_humor
14. be.beauty
15. thesmolny
16. femalemem
17. bot_maxim
18. psyxov
19. team
20. i_d_t


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [218]:

X, y = synthetic_dataset(1000)

clf = CatBoostClassifier( 
                n_estimators = 5000,
                learning_rate= 0.01,
                loss_function='MultiClass',
                random_seed= 42,
                metric_period=1000,
                depth= 8)
clf.fit(X, y)

pred = clf.predict(X).reshape(1, -1)

print("Accuracy:", np.mean(pred == y))
print("Importance:", clf.get_feature_importance())

0:	learn: 1.0793979	total: 649us	remaining: 3.25s
1000:	learn: 0.0027530	total: 876ms	remaining: 3.5s
2000:	learn: 0.0012342	total: 1.65s	remaining: 2.47s
3000:	learn: 0.0008980	total: 2.44s	remaining: 1.62s
4000:	learn: 0.0007048	total: 3.82s	remaining: 953ms
4999:	learn: 0.0005796	total: 4.96s	remaining: 0us
Accuracy: 1.0
Importance: [5.60552241e-03 6.56860075e-03 2.78576260e+01 2.78160349e+01
 4.43106874e+01 3.47751298e-03]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [194]:
X, y_age, y_sex, features = read_dataset("/content/drive/MyDrive/vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [219]:
clf = CatBoostClassifier(
                n_estimators = 5000,
                learning_rate= 0.01,
                loss_function='MultiClass',
                random_seed= 42,
                metric_period=1000,
                depth= 8)
clf.fit(X_train, y_age_train, eval_set=(X_eval, y_age_eval),
            cat_features=np.arange(X.shape[1]),
            use_best_model=True,
            verbose=True)

pred = clf.predict(X_test).reshape(1, -1)

print("Accuracy:", np.mean(pred == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(clf.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

0:	learn: 1.0945574	test: 1.0950794	best: 1.0950794 (0)	total: 130ms	remaining: 10m 49s
1000:	learn: 0.5532383	test: 0.6444955	best: 0.6444955 (1000)	total: 1m 27s	remaining: 5m 49s
2000:	learn: 0.4497911	test: 0.6149536	best: 0.6149536 (2000)	total: 2m 50s	remaining: 4m 16s
3000:	learn: 0.3865771	test: 0.6058500	best: 0.6058500 (3000)	total: 4m 14s	remaining: 2m 49s
4000:	learn: 0.3396047	test: 0.6036602	best: 0.6036602 (4000)	total: 5m 37s	remaining: 1m 24s
4999:	learn: 0.3033387	test: 0.6031478	best: 0.6031478 (4999)	total: 6m 59s	remaining: 0us

bestTest = 0.6031478377
bestIteration = 4999

Accuracy: 0.7364438839848676
Most important features:
1. mudakoff
2. ovsyanochan
3. 4ch
4. dayvinchik
5. styd.pozor
6. leprum
7. rhymes
8. rapnewrap
9. kino_mania
10. fuck_humor


#### Пол

In [220]:
clf = CatBoostClassifier(
                n_estimators = 5000,
                learning_rate= 0.01,
                loss_function='MultiClass',
                random_seed= 42,
                metric_period=1000,
                depth= 8)

clf.fit(X_train, y_sex_train, eval_set=(X_eval, y_sex_eval),
            cat_features=np.arange(X.shape[1]),
            use_best_model=True,
            verbose=True)

pred = clf.predict(X_test).reshape(1, -1)

print("Accuracy:", np.mean(pred == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(clf.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

0:	learn: 0.6903986	test: 0.6904889	best: 0.6904889 (0)	total: 63.4ms	remaining: 5m 17s
1000:	learn: 0.3081333	test: 0.3419165	best: 0.3419165 (1000)	total: 1m 2s	remaining: 4m 11s
2000:	learn: 0.2496829	test: 0.3150341	best: 0.3150341 (2000)	total: 2m 5s	remaining: 3m 8s
3000:	learn: 0.2141910	test: 0.3059229	best: 0.3059229 (3000)	total: 3m 8s	remaining: 2m 5s
4000:	learn: 0.1885220	test: 0.3018044	best: 0.3018044 (4000)	total: 4m 11s	remaining: 1m 2s
4999:	learn: 0.1683847	test: 0.3004953	best: 0.3004953 (4999)	total: 5m 14s	remaining: 0us

bestTest = 0.3004952582
bestIteration = 4999

Accuracy: 0.8663303909205549
Most important features:
1. 40kg
2. mudakoff
3. girlmeme
4. modnailru
5. academyofman
6. team
7. rapnewrap
8. i_d_t
9. femalemem
10. igm
